In [ ]:
pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from faker import Faker

# Initialize Faker for random word generation
fake = Faker()

# Define the number of rows and sensors
num_rows = 100000
num_sensors = 6  # Number of sensor values to generate

# Generate random words and sensor values
data = []
for _ in range(num_rows):
    # Use fake.word() to generate random words (not necessarily unique)
    word = fake.word()
    sensor_values = np.random.uniform(0, 1, num_sensors).tolist()  # Generate 6 random sensor values
    data.append([word] + sensor_values)

# Create a DataFrame
columns = ["word"] + [f"sensor_{i+1}" for i in range(num_sensors)]
df = pd.DataFrame(data, columns=columns)

# Save to CSV
df.to_csv("fake_sign_language_dataset.csv", index=False)
print("Fake dataset saved to 'fake_sign_language_dataset.csv'")

Fake dataset saved to 'fake_sign_language_dataset.csv'


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the dataset
def load_dataset(file_path):
    """
    Loads the dataset from a CSV file.
    Args:
        file_path (str): Path to the CSV file.
    Returns:
        X (np.array): Sensor data of shape (samples, 15).
        y (np.array): Word labels of shape (samples,).
    """
    df = pd.read_csv(file_path)
    X = df.iloc[:, 1:].values  # Sensor values (columns 1 to 15)
    y = df.iloc[:, 0].values   # Word labels (column 0)
    return X, y

# Preprocess the data
def preprocess_data(X, y):
    """
    Preprocesses the data by normalizing sensor values and encoding word labels.
    Args:
        X (np.array): Sensor data of shape (samples, 15).
        y (np.array): Word labels of shape (samples,).
    Returns:
        X_train, X_test, y_train, y_test: Split and preprocessed data.
    """
    # Normalize sensor values
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X)

    # Encode word labels as integers
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, label_encoder

# Define the Gesture-to-Word Model
def build_gesture_model(input_shape, num_classes):
    """
    Builds a neural network model to map sensor data to words.
    Args:
        input_shape (int): Number of input features (15 sensor values).
        num_classes (int): Number of output classes (words).
    Returns:
        model: Compiled Keras model.
    """
    model = models.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # Output layer for word classification
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Main execution
if __name__ == "__main__":
    # Load the dataset
    file_path = "fake_sign_language_dataset.csv"
    X, y = load_dataset(file_path)

    # Preprocess the data
    X_train, X_test, y_train, y_test, label_encoder = preprocess_data(X, y)

    # Build the model
    num_classes = len(label_encoder.classes_)  # Number of unique words
    gesture_model = build_gesture_model(input_shape=15, num_classes=num_classes)

    # Train the model
    gesture_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    # Save the model
    gesture_model.save("gesture_to_word_model.h5")
    print("Gesture-to-Word Model saved.")

Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 7.5533e-04 - loss: 6.8818 - val_accuracy: 8.0000e-04 - val_loss: 6.8799
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.0011 - loss: 6.8748 - val_accuracy: 9.0000e-04 - val_loss: 6.8843
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.0019 - loss: 6.8605 - val_accuracy: 0.0011 - val_loss: 6.8922
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.0022 - loss: 6.8334 - val_accuracy: 7.0000e-04 - val_loss: 6.9192
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0024 - loss: 6.8024 - val_accuracy: 9.5000e-04 - val_loss: 6.9443
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0033 - loss: 6.7714 - val_accuracy: 9.5000e-04 - val_loss: 6.9619
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.0037 - loss: 6.7461 - val_accuracy: 9.0000e-04 - val_loss: 6.9873
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - 

Gesture-to-Word Model saved.


In [ ]:
test_loss, test_accuracy = gesture_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 6.8376e-04 - loss: 7.0647
Test Accuracy: 0.0008


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from faker import Faker

# # Step 1: Generate Fake Dataset
# def generate_fake_dataset():
#     """
#     Generates a fake dataset with 100,000 words and 15 sensor values each.
#     """
#     fake = Faker()
#     num_rows = 100000
#     num_sensors = 15
#     data = []
#     for word_id in range(num_rows):
#         word = fake.unique.word()  # Generate a unique random word
#         base_pattern = np.sin(np.linspace(0, 2 * np.pi * (word_id % 10), num_sensors))  # Unique pattern for each word
#         noise = np.random.uniform(-0.1, 0.1, num_sensors)  # Add some noise
#         sensor_values = base_pattern + noise  # Combine base pattern and noise
#         data.append([word] + sensor_values.tolist())  # Add word and sensor values to the dataset
#     columns = ["word"] + [f"sensor_{i+1}" for i in range(num_sensors)]  # Column names
#     df = pd.DataFrame(data, columns=columns)  # Create a DataFrame
#     df.to_csv("fake_sign_language_dataset.csv", index=False)  # Save to CSV
#     print("Fake dataset saved to 'fake_sign_language_dataset.csv'")

# Step 2: Train the Gesture-to-Word Model
def train_gesture_to_word_model():
    """
    Trains a neural network to map sensor data to words.
    """
    # Load dataset
    df = pd.read_csv("fake_sign_language_dataset.csv")
    X = df.iloc[:, 1:].values  # Sensor values (columns 1 to 15)
    y = df.iloc[:, 0].values   # Word labels (column 0)

    # Preprocess data
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X)  # Normalize sensor values
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Encode words as integers
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=42)  # Split data

    # Build and train model
    num_classes = len(label_encoder.classes_)  # Number of unique words
    model = models.Sequential([
        layers.Input(shape=(15,)),  # Input layer for 15 sensor values
        layers.Dense(128, activation='relu'),  # Hidden layer 1
        layers.Dropout(0.2),  # Dropout for regularization
        layers.Dense(64, activation='relu'),  # Hidden layer 2
        layers.Dense(num_classes, activation='softmax')  # Output layer for word classification
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Compile model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))  # Train model

    # Save model and preprocessing objects
    model.save("gesture_to_word_model.h5")  # Save the trained model
    np.save("label_encoder_classes.npy", label_encoder.classes_)  # Save label encoder classes
    np.save("scaler_mean.npy", scaler.mean_)  # Save scaler mean
    np.save("scaler_scale.npy", scaler.scale_)  # Save scaler scale
    print("Gesture-to-Word Model saved.")

# Step 3: Gesture-to-Sentence Pipeline
def gesture_to_sentence():
    """
    Captures a sequence of gestures and forms a sentence.
    """
    # Load model and preprocessing objects
    model = tf.keras.models.load_model("gesture_to_word_model.h5")  # Load the trained model
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load("label_encoder_classes.npy", allow_pickle=True)  # Load label encoder classes
    scaler = StandardScaler()
    scaler.mean_ = np.load("scaler_mean.npy")  # Load scaler mean
    scaler.scale_ = np.load("scaler_scale.npy")  # Load scaler scale

    # Function to predict a word
    def predict_word(sensor_data):
        """
        Predicts the word from sensor data.
        """
        sensor_data_normalized = scaler.transform(sensor_data)  # Normalize sensor data
        predicted_index = np.argmax(model.predict(sensor_data_normalized), axis=1)  # Predict word index
        word = label_encoder.inverse_transform(predicted_index)[0]  # Convert index to word
        return word

    # Capture gestures and form a sentence
    sentence = []  # List to store predicted words
    print("Start making gestures. Type 'done' when finished.")
    while True:
        user_input = input("Press Enter to capture a gesture or type 'done': ")  # Simulate user input
        if user_input.lower() == "done":
            break
        sensor_data = np.random.uniform(0, 1, (1, 15))  # Simulate sensor data (replace with real data)
        print("Sensor data captured:", sensor_data)
        word = predict_word(sensor_data)  # Predict the word
        print(f"Predicted word: {word}")
        sentence.append(word)  # Add the word to the sentence
    final_sentence = " ".join(sentence)  # Stitch words into a sentence
    print("\nFinal Sentence:", final_sentence)

# Main execution
if __name__ == "__main__":
    # Step 1: Generate fake dataset (run once)
    # generate_fake_dataset()

    # Step 2: Train the model (run once)
    # train_gesture_to_word_model()

    # Step 3: Run the Gesture-to-Sentence pipeline
    gesture_to_sentence()

FileNotFoundError: [Errno 2] No such file or directory: 'label_encoder_classes.npy'